In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import *
from sklearn.preprocessing import*
import datetime,time
from sklearn.metrics import*
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from skopt import BayesSearchCV
import re
from skopt.callbacks import DeltaXStopper

pd.options.display.float_format = '{:.7f}'.format
pd.set_option('display.max_columns', None)

In [2]:
df= pd.read_csv('D:/project/homecredit/final/v3/final_all_onehotencoding_v3.csv',index_col=0)

In [3]:
df = df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x.replace(' ', '')))

X = df.drop(columns=['SK_ID_CURR', 'TARGET'])
# X.to_csv('D:/project/homecredit/final/v3/final_all_onehotencoding_formodel_v3.csv')
y = df['TARGET']


In [4]:
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,AGE,JOB_TENURE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_EMPLOYED_365243,CNT_INSTALMENT_median,Contract_Change_rateCNT_INSTALMENT_mean,DAYS_CREDIT_ENDDATE_max,AMT_PAYMENT_MAXANNUITY_max,Contract_Change_rateCNT_INSTALMENT,DAYS_CREDIT_UPDATE_max,countSK_ID_BUREAUCREDIT_ACTIVE_Active_count,3_DAYS_ENTRY_DIFF_MEAN_mean,AMT_PAYMENT_MAX_median,AMT_PAYMENT_MAXCREDIT_max,AMT_PAYMENT_MAXCREDIT_median,DAYS_CREDIT_ENDDATE_min,CNT_INSTALMENT_max,AMT_CREDIT_SUM_DEBT_mean,Contract_Change_count,AMT_CREDIT_SUM_OVERDUE_max,AMT_CREDIT_SUM_max,PAYMENT_MAXMEDIAN_median,Contract_Change_Count_mean,AMT_PAYMENT_MEDIAN_median,DAYS_ENTRY_DIFF_MAX_max,3_DAYS_ENTRY_DIFF_MAX_mean,AMT_PAYMENT_MAX_max,6_DAYS_ENTRY_DIFF_MAX_mean,MAX_OVERDUECREDIT_mean,CNT_INSTALMENTTotal_Months_rate,PREV_DAYS_TERMINATION_min,CNT_INSTALMENTTotal_Months_rate_mean,9_DELAY_mean,PREV_NAME_YIELD_GROUP_XNA_rate,MONTHS_BALANCE_finish,PREV_GOODS_PRICECREDIT_min,OUTSTANDING_sum,DAYS_CREDIT_recent,AMT_PAYMENT_ALL_mean,countSK_ID_BUREAUCREDIT_TYPE_Microloan_count,DELAY_mean,DAYS_CREDIT_ENDDATE_median,MONTHS_BALANCE_start,AMT_CREDIT_SUM_OVERDUE_mean,DAYS_CREDIT_UPDATE_median,DAYS_ENTRY_DIFF_MEAN_mean,PREV_DAYS_LAST_DUE_min,9_DAYS_ENTRY_DIFF_MAX_mean,DAYS_CREDIT_mean,DEBTCREDIT_mean,DAYS_ENTRY_DIFF_MAX_median,DEBTCREDIT_max,AMT_ANNUITY_CASH_LOANS_MEAN,AMT_APPLICATION_CASH_LOANS_MEAN,AMT_CREDIT_CASH_LOANS_MEAN,AMT_GOODS_PRICE_CASH_LOANS_MEAN,AMT_ANNUITY_CONSUMER_LOANS_MEAN,AMT_APPLICATION_CONSUMER_LOANS_MEAN,AMT_CREDIT_CONSUMER_LOANS_MEAN,AMT_GOODS_PRICE_CONSUMER_LOANS_MEAN,AMT_ANNUITY_REVOLVING_LOANS_MEAN,AMT_APPLICATION_REVOLVING_LOANS_MEAN,AMT_CREDIT_REVOLVING_LOANS_MEAN,AMT_GOODS_PRICE_REVOLVING_LOANS_MEAN,creditcard_high,creditcard_normal,creditcard_low,instalments_overdue_DAY_mean,instalments_overdue_AMT_mean,instalments_PERIOD,BUREAU_DAYS_CREDIT,BUREAU_DAYS_CREDIT_ENDDATE,BUREAU_DAYS_ENDDATE_FACT,CDCS_ratio,bureau_annuity_mean,bereau_balance_status,NAME_CONTRACT_TYPE_Revolvingloans,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Groupofpeople,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Spousepartner,NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercialassociate,NAME_INCOME_TYPE_Maternityleave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Stateservant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Highereducation,NAME_EDUCATION_TYPE_Incompletehigher,NAME_EDUCATION_TYPE_Lowersecondary,NAME_EDUCATION_TYPE_Secondarysecondaryspecial,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Singlenotmarried,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Houseapartment,NAME_HOUSING_TYPE_Municipalapartment,NAME_HOUSING_TYPE_Officeapartment,NAME_HOUSING_TYPE_Rentedapartment,NAME_HOUSING_TYPE_Withparents,OCCUPATION_TYPE_Cleaningstaff,OCCUPATION_TYPE_Cookingstaff,OCCUPATION_TYPE_Corestaff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HRstaff,OCCUPATION_TYPE_Highskilltechstaff,OCCUPATION_TYPE_ITstaff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_LowskillLaborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicinestaff,OCCUPATION_TYPE_Privateservicestaff,OCCUPATION_TYPE_Realtyagents,OCCUPATION_TYPE_Salesstaff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Securitystaff,OCCUPATION_TYPE_Waitersbarmenstaff,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGA

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test= scaler.transform(X_test)


In [8]:
params = {
    'learning_rate': (0.01, 0.02,0.03, ),  
    'num_leaves': (31, 34),
    'colsample_bytree': (0.8, 1.0),
    'subsample': (0.8, 1.0),
    'max_depth': (5, 10),
    'reg_alpha': (0.01, 0.05, 'log-uniform'),  
    'reg_lambda': (0.05, 0.1, 'log-uniform'),  
    'min_split_gain': (0.01, 0.03),
    'min_child_weight': (38, 40)
}


lgb_params = {
    'objective': 'binary',
    'is_unbalance': True,
    'device': 'gpu',
    'learning_rate': 0.03,
    'n_iter': 3000,
    'eval_metric': 'auc',  
}

lgb_classifier = lgb.LGBMClassifier(**lgb_params)

# Create BayesSearchCV instance
bayes = BayesSearchCV(
    lgb_classifier, params, n_iter=50, scoring='roc_auc', cv=5, n_jobs=4
)


start_time = time.time()
bayes.fit(X_train, y_train)
best_params = bayes.best_params_
best_score = bayes.best_score_
print("Best parameters found: ", bayes.best_params_)
print("Best score found: ", bayes.best_score_)

end_time = time.time()
execution_time = end_time - start_time

print("程式執行花費的时间：", round(execution_time, 2), "秒")
print("完成时间:", datetime.datetime.now())


